# 0 Begin chapter

## 0.1. install

In [3]:
!python -m pip install pandas pyarrow

  Obtaining dependency information for fastparquet from https://files.pythonhosted.org/packages/d7/df/6f86eb325f7a50c63ba4bfcbd5e1dfc75969dd760163af0527e7f382f16b/fastparquet-2023.8.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cramjam>=2.3 from https://files.pythonhosted.org/packages/69/e5/261a0baa524bf9e5c1e7a62e33ad8ccc0a9c40ccc6ac1c323f9ded203cf4/cramjam-2.7.0-cp311-none-win_amd64.whl.metadata
  Obtaining dependency information for fsspec from https://files.pythonhosted.org/packages/fe/d3/e1aa96437d944fbb9cc95d0316e25583886e9cd9e6adc07baad943524eda/fsspec-2023.9.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/715.7 kB ? eta -:--:--
    --------------------------------------- 10.2/715.7 kB ? eta -:--:--
   --------- ------------------------------ 163.8/715.7 kB 2.4 MB/s eta 0:00:01
   ------------------------------------- -- 665.6/715.7 kB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 715.7/715.7 kB 6.5 MB/

## 0.2. Import

In [212]:
import pandas as pd
import numpy as np
import sys
import json

In [213]:
sys.version
# '3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]'

'3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]'

## 0.3. convert csv to parquet

In [4]:
# hotels = pd.read_csv('data/hotels.csv')
# hotels.to_parquet('data/hotels_parquet.gzip', engine='pyarrow', compression='gzip')

## 0.4. Functions

In [10]:
def _get_col_names(df: pd.DataFrame, max_row_info: int=8) -> pd.DataFrame:
    ostatok = df.shape[1] % max_row_info
    col = df.shape[1] // max_row_info
    
    new_list = list()
    for index in range(col if not ostatok else col + 1):
        start = index*max_row_info
        end = (index + 1)*max_row_info
        new_list.append(df.columns[start:end].to_list())
    return pd.DataFrame((new_list))

In [200]:
hotels = pd.read_parquet('data/hotels_parquet.gzip')
_get_col_names(hotels,3)

,0,1,2
0,hotel_address,additional_number_of_scoring,review_date
1,average_score,hotel_name,reviewer_nationality
2,negative_review,review_total_negative_word_counts,total_number_of_reviews
3,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given
4,reviewer_score,tags,days_since_review
5,lat,lng,None


---
- hotel_address — адрес отеля;
- review_date — дата, когда рецензент разместил соответствующий отзыв;
- average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
- hotel_name — название отеля;
- reviewer_nationality — страна рецензента;
- negative_review — отрицательный отзыв, который рецензент дал отелю;
- review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
- positive_review — положительный отзыв, который рецензент дал отелю;
- review_total_positive_word_counts — общее количество слов в положительном отзыве.
- reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
- total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
- total_number_of_reviews — общее количество действительных отзывов об отеле;
- tags — теги, которые рецензент дал отелю;
- days_since_review — количество дней между датой проверки и датой очистки;
- additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
- lat — географическая широта отеля;
- lng — географическая долгота отеля.
---

# 1. Исследование данных

## 1.1. Отделение Категориальных признаков

In [201]:
resolution = hotels.shape[0]
tmp_cols = (
            hotels
            .nunique()
            .sort_values(ascending=False)
            .to_frame(name='count')
            .assign(frec=lambda x: round(100*x['count']/resolution).astype('UInt8'))
            )

cat_cols = (
            tmp_cols
            .query('frec < 20')
            .index
            .to_list()
            )

other_cols = (
                tmp_cols
                .query('frec >= 20')
                .index
                .to_list()
                )

print(f'cat_cols: {len(cat_cols)}; other_cols: {len(other_cols)}')


cat_cols: 15; other_cols: 2


## 1.2. Преобразование Категориальных признаков

## 1.2.1. Преобразование типов Object

In [202]:
object_cols = hotels[cat_cols].select_dtypes(include='object').columns.to_list()
# other_cat_cols = [x for x in cat_cols if x not in object_cols]
other_cat_cols = list(filter(lambda x: x not in object_cols, cat_cols))

hotels[object_cols].head()

,tags,hotel_address,hotel_name,days_since_review,review_date,reviewer_nationality
0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",Stratton Street Mayfair Westminster Borough Lo...,The May Fair Hotel,531 day,2/19/2016,United Kingdom
1,"[' Business trip ', ' Couple ', ' Standard Dou...",130 134 Southampton Row Camden London WC1B 5AF...,Mercure London Bloomsbury Hotel,203 day,1/12/2017,United Kingdom
2,"[' Leisure trip ', ' Solo traveler ', ' Modern...",151 bis Rue de Rennes 6th arr 75006 Paris France,Legend Saint Germain by Elegancia,289 day,10/18/2016,China
3,"[' Leisure trip ', ' Solo traveler ', ' Standa...",216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,Mercure Paris 19 Philharmonie La Villette,681 day,9/22/2015,United Kingdom
4,"[' Business trip ', ' Couple ', ' Standard Dou...",Molenwerf 1 1014 AG Amsterdam Netherlands,Golden Tulip Amsterdam West,516 day,3/5/2016,Poland


In [203]:
# Преобразование столбца "tags"
hotels['tags'] = hotels['tags'].str.replace('\'','\"' ).apply(lambda x: json.loads(x))#.explode(ignore_index=True).str.strip().nunique()

# Преобразование признака "days_since_review"
hotels['days_since_review'] = hotels['days_since_review'].str.extract('(\d+) day', expand=False).astype('UInt16')

# Преобразование признака "review_date"
hotels['review_date'] = pd.to_datetime(hotels['review_date'], format='%d/%M/%Y').max()

# Преобразование признака  "reviewer_nationality"
hotels['hotel_name'] = hotels['hotel_name'].str.strip().replace('', np.nan)
category_order_list = sorted([x for x in hotels['hotel_name'].unique() if not pd.isnull(x)])
hotels['hotel_name'] = hotels['hotel_name'].astype('category').cat.set_categories(category_order_list, ordered=True)

# Преобразование признака  "reviewer_nationality"
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].str.strip().replace('', np.nan)
category_order_list = sorted([x for x in hotels['reviewer_nationality'].unique() if not pd.isnull(x)])
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].astype('category').cat.set_categories(category_order_list, ordered=True)

# Преобразование признака "hotel_address"
hotels['hotel_address'] = hotels['hotel_address'].astype('string')

## 1.2.1. Преобразование типов других типов

In [204]:
hotels[other_cat_cols].head()

,lat,lng,total_number_of_reviews,additional_number_of_scoring,review_total_negative_word_counts,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,average_score
0,51.507894,-0.143671,1994,581,3,4,7,10.0,8.4
1,51.521009,-0.123097,1361,299,3,2,14,6.3,8.3
2,48.845377,2.325643,406,32,6,0,14,7.5,8.9
3,48.888697,2.394540,607,34,0,11,8,10.0,7.5
4,52.385601,4.847060,7586,914,4,20,10,9.6,8.5


In [205]:
hotels['lat'] = hotels['lat'].astype('Float32')
hotels['lng'] = hotels['lng'].astype('Float32')
hotels['total_number_of_reviews'] = hotels['total_number_of_reviews'].astype('UInt16')
hotels['additional_number_of_scoring'] = hotels['additional_number_of_scoring'].astype('UInt16')
hotels['review_total_negative_word_counts'] = hotels['review_total_negative_word_counts'].astype('UInt16')
hotels['review_total_positive_word_counts'] = hotels['review_total_positive_word_counts'].astype('UInt16')
hotels['total_number_of_reviews_reviewer_has_given'] = hotels['total_number_of_reviews_reviewer_has_given'].astype('UInt16')
hotels['reviewer_score'] = hotels['reviewer_score'].astype('Float32')
hotels['average_score'] = hotels['average_score'].astype('Float32')

In [206]:
hotels[other_cat_cols].head()

,lat,lng,total_number_of_reviews,additional_number_of_scoring,review_total_negative_word_counts,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,average_score
0,51.507893,-0.143671,1994,581,3,4,7,10.0,8.4
1,51.521008,-0.123097,1361,299,3,2,14,6.3,8.3
2,48.845379,2.325643,406,32,6,0,14,7.5,8.9
3,48.888699,2.39454,607,34,0,11,8,10.0,7.5
4,52.385601,4.84706,7586,914,4,20,10,9.6,8.5


In [207]:
hotels['negative_review'] = hotels['negative_review'].astype('string')
hotels['positive_review'] = hotels['positive_review'].astype('string')

In [208]:
hotels = hotels.explode('tags', ignore_index=True)
hotels['tags'] = hotels['tags'].str.strip()
category_order_list = sorted([x for x in hotels['tags'].unique() if not pd.isnull(x)])
hotels['tags'] = hotels['tags'].astype('category').cat.set_categories(category_order_list, ordered=True)

In [209]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1767522 entries, 0 to 1767521
Data columns (total 17 columns):
 #   Column                                      Dtype         
---  ------                                      -----         
 0   hotel_address                               string        
 1   additional_number_of_scoring                UInt16        
 2   review_date                                 datetime64[ns]
 3   average_score                               Float32       
 4   hotel_name                                  category      
 5   reviewer_nationality                        category      
 6   negative_review                             string        
 7   review_total_negative_word_counts           UInt16        
 8   total_number_of_reviews                     UInt16        
 9   positive_review                             string        
 10  review_total_positive_word_counts           UInt16        
 11  total_number_of_reviews_reviewer_has_given  UInt16

In [210]:
hotels.to_parquet('data/hotels_v2_parquet.gzip', engine='pyarrow', compression='gzip')

In [211]:
hotels = pd.read_parquet('data/hotels_v2_parquet.gzip')

---

In [198]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [11]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [12]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


ValueError: could not convert string to float: 'Pla a de Llevant s n Sant Mart 08019 Barcelona Spain'

In [9]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.14138286324223787


Небольшой бонус:


In [10]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
# hotels = hotels.fillna(0)